In [7]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import glob, os, sys, re, math, random
import xml.etree.ElementTree as ET
from rules import *
from predict import *
from raven_data import load_question, display_problem
from tqdm.notebook import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
path = '../RAVEN/new_data_0'
subtype = '*'
items = sorted([x.split('.xml')[0] for x in glob.glob(os.path.join(path, subtype, '*.xml'))])
debug=False
raw_X = []
raw_y = []
raw_partition = []
if debug:
    items=['../RAVEN/new_data_1/distribute_four/RAVEN_620_train']
for index, item in tqdm(enumerate(items), total=len(items)):
    embeddings, embedding_names, answer = load_question(item, display=debug, debug=False)
    if any(len(embedding) != len(embeddings[0]) for embedding in embeddings):
        print("Error!", item, " has embeddings of different lengths")
        break
    guess, _, y_entry = predict(embeddings,embedding_names,debug=debug)
    x = np.array(embeddings[:8])
    if guess == answer:
        raw_X.append(x)
        raw_y.append(y_entry)
        raw_partition.append([partition_type in item for partition_type in ['train', 'val', 'test']])
    else:
        if debug:
            grid = [[0, 1, 2], [3, 4, 5], [6, 7, None]]
            print("\n".join([str([embeddings[col] if col is not None else None for col in row ]) for row in grid]))
            print("Answer", answer, embeddings[8+answer], "Guess", guess, embeddings[8+guess] if guess is not None else None)
        print("WRONG",item, "Wrong guess", guess, "Correct answer", answer)
print("Using", len(raw_X), "datapoints")

  0%|          | 0/700 [00:00<?, ?it/s]

Using 700 datapoints


In [9]:
max_length = max([x.shape[1] for x in raw_X])
def pad(x):
    return np.pad(x, ((0,0), (0, max_length - x.shape[1])), mode='constant').flatten()
X = np.array([pad(x) for x in raw_X])
y = np.array(raw_y)
partition = np.array(raw_partition)
X.shape,y.shape

((700, 112), (700, 5))

In [10]:
# Split data across train/val/test partitions
X_train, X_val, X_test = X[partition[:, 0]], X[partition[:, 1]], X[partition[:, 2]]
y_train, y_val, y_test = y[partition[:, 0]], y[partition[:, 1]], y[partition[:, 2]]
X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

((420, 112), (140, 112), (140, 112), (420, 5), (140, 5), (140, 5))

In [13]:
base_model = RandomForestClassifier()
model = MultiOutputClassifier(base_model)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.92      0.92       119
           1       0.64      0.91      0.75        88
           2       0.63      0.49      0.55        65
           3       0.73      0.89      0.80        94
           4       0.77      0.83      0.80        41

   micro avg       0.75      0.83      0.79       407
   macro avg       0.74      0.81      0.77       407
weighted avg       0.76      0.83      0.79       407
 samples avg       0.75      0.82      0.76       407



/home/alex/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [19]:
# Use model predictions

path = '../RAVEN/new_data_0'
subtype = '*'
items = sorted([x.split('.xml')[0] for x in glob.glob(os.path.join(path, subtype, '*.xml'))])
correct = 0
debug=True
if debug:
    items=['../RAVEN/new_data_0/distribute_four/RAVEN_62_train']
for item in items:
    embeddings, embedding_names, answer = load_question(item, display=False, debug=False)
    if any(len(embedding) != len(embeddings[0]) for embedding in embeddings):
        print("Error!", item, " has embeddings of different lengths")
        break
    x = pad(np.array(embeddings[:8])).reshape(1, -1)
    y_pred = model.predict(x)
    y_prob = model.predict_proba(x)
    y_prob_formatted = [y_prob_i[0][1] for y_prob_i in y_prob]
    print({rule.__name__: prob for rule, prob in zip(rule_order, y_prob_formatted)})
    guess, _, y_entry = predict(embeddings,embedding_names,debug=False,prob=y_prob_formatted)
    print(y_entry)
    if guess == answer:
        correct += 1
    else:
        if debug:
            grid = [[0, 1, 2], [3, 4, 5], [6, 7, None]]
            print("\n".join([str([embeddings[col] if col is not None else None for col in row ]) for row in grid]))
            print("Answer", answer, embeddings[8+answer], "Guess", guess, embeddings[8+guess] if guess is not None else None)
        print("WRONG",item, "Wrong guess", guess, "Correct answer", answer)
print("Final score:", correct, "out of", len(items))

{'constant': 0.13, 'progression': 0.87, 'arithmetic': 0.83, 'distribute_three': 0.11, 'noise': 0.85}
[0, 1, 1, 0, 1]
Final score: 1 out of 1
